In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from urllib.request import urlopen
import urllib.request
from datetime import datetime
from tqdm import tqdm
import time
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
hrefs = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 

In [6]:
def crawl(url):
    while True:
        try:
            r = requests.get(url,headers = headers)
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(r.text, "html.parser")

    div = soup.find_all('h3', attrs = {"class":"elementor-post__title"})
    
    if len(div) == 0:
        return

    for link in soup.find_all('h3', attrs = {"class":"elementor-post__title"}):

        href = link.find('a').get('href')

        hrefs.append(href)

In [7]:
max_worker = 5

links = ['https://ikram.org.my/artikel/']


for url in links:
    
    print(url)
    r = requests.get(url,headers=headers)
    soup = BeautifulSoup(r.content, "html.parser")
    a = soup.find_all('a', attrs = {"class":"page-numbers"})
    a = [a_.get('href') for a_ in a if a_.get('href')]
    pgno = [a_ for a_ in a if 'page' in a_]
    max_page = max([int(a_.split('page/')[1].replace('/','')) for a_ in pgno])
    print(max_page)

    
    for i in tqdm(range(1, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{url}page/{a}' for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url): url for url in urls}
        
        for future in as_completed(futures):
            future.result()

https://ikram.org.my/artikel/
373


100%|██████████| 75/75 [06:47<00:00,  5.43s/it]


In [11]:
hrefs = list(set(hrefs))

In [12]:
len(hrefs)

3388

In [13]:
with open(f'ikram-link.json', 'a') as f:
    json.dump(hrefs, f)

In [8]:
with open('ikram-link.json') as fopen:
    hrefs = json.load(fopen)

In [18]:
def process_url(x):
    
    while True:
        try:
            r = requests.get(x, headers=headers)
            
            if r.status_code == 508:
                time.sleep(20)
                continue
                
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "html.parser")

    try:
        headline = soup.find('h1', class_="elementor-heading-title elementor-size-default").text
        h = soup.find('div', class_="elementor-widget-wrap elementor-element-populated")
        p = [p.text for p in h.find_all('p') if p.text and len(p.text.split()) > 5]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None

    
    data = {'url': x, 'headline': headline, 'content': p}
    
    return data

In [19]:
process_url("https://ikram.org.my/kemelut-sudan/")

{'url': 'https://ikram.org.my/kemelut-sudan/',
 'headline': 'Rampasan kuasa Sudan: Kemelut semakin larut',
 'content': ['Jawatankuasa penentangan Sudan menzahirkan keraguan terhadap inisiatif Pertubuhan Bangsa-Bangsa Bersatu (PBB) yang ingin mencari penyelesaian terhadap krisis politik di negara itu.',
  '“Sebarang inisiatif yang tidak termasuk penyingkiran tentera daripada kuasa adalah tidak dialu-alukan,” kata Jurucakap jawatankuasa itu.',
  'Masih tiada kepimpinan yang bersatu untuk jawatankuasa penentangan yang menganjurkan protes popular terhadap rampasan kuasa tentera, tetapi mereka sebulat suara menyeru kepada pemerintahan awam tanpa kerjasama dengan tentera.',
  'Bagaimanapun, sesetengah pihak percaya bahawa peluang kejayaan inisiatif antarabangsa itu mungkin besar, memandangkan parti politik mengalu-alukannya.',
  'Utusan PBB ke Sudan, Volker Peretz mengumumkan dalam sidang medianya Isnin lalu dengan berkata;',
  '“Menjadi hasrat PBB bagi melancarkan dialog antara parti-parti 

In [20]:
max_workers = 3

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in hrefs]
    
    for future in tqdm(futures, total=len(hrefs)):
        result = future.result()
        if result:
            with open('ikram.jsonl', 'a') as f:
                json.dump(result, f)
                f.write('\n')

  0%|          | 10/3388 [00:08<42:23,  1.33it/s] 

error in link:https://ikram.org.my/earthquake-in-nepal-ikram-extends-condolences/
'NoneType' object has no attribute 'text'


100%|██████████| 3388/3388 [47:40<00:00,  1.18it/s] 
